<a href="https://colab.research.google.com/github/Bhavya418/quant_alphas/blob/main/Harami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install plotly
!pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt
import requests
import plotly.graph_objects as go
from datetime import datetime

In [4]:
def fetch_daily_ohlc(symbol, apikey):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=compact&apikey={apikey}"
    r = requests.get(url)
    return r.json()

In [49]:
apikey = "SHEYWPL1ZYCDZ17D"
symbol = "RPOWER.BSE"
data = fetch_daily_ohlc(symbol, apikey)
print(data)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'RPOWER.BSE', '3. Last Refreshed': '2025-04-23', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-04-23': {'1. open': '43.8900', '2. high': '44.1800', '3. low': '42.3200', '4. close': '43.5500', '5. volume': '3519503'}, '2025-04-22': {'1. open': '44.2600', '2. high': '44.8900', '3. low': '43.2000', '4. close': '43.5400', '5. volume': '4283262'}, '2025-04-21': {'1. open': '42.2800', '2. high': '44.6500', '3. low': '42.0500', '4. close': '44.2100', '5. volume': '8797336'}, '2025-04-17': {'1. open': '41.9900', '2. high': '43.0000', '3. low': '41.6700', '4. close': '42.1400', '5. volume': '3834126'}, '2025-04-16': {'1. open': '41.8600', '2. high': '42.5500', '3. low': '41.4700', '4. close': '41.8100', '5. volume': '5141672'}, '2025-04-15': {'1. open': '40.0600', '2. high': '42.6000', '3. low': '40.0600', '4. close': '41.6300', '5. volume': '6385477'}, 

In [6]:
def is_downtrend(candles):
    """True if lows are decreasing = downtrend"""
    low_prices = [float(c["3. low"]) for c in candles]
    for earlier, later in zip(low_prices, low_prices[1:]):
        if earlier <= later:
            return False
    return True

def is_uptrend(candles):
    """True if lows are increasing = uptrend"""
    low_prices = [float(c["3. low"]) for c in candles]

    for earlier, later in zip(low_prices, low_prices[1:]):
        if earlier >= later:
            return False
    return True

In [38]:
def is_bullish_harami(prev_candle, current_candle):
    prev_open = float(prev_candle["1. open"])
    prev_close = float(prev_candle["4. close"])
    current_open = float(current_candle["1. open"])
    current_close = float(current_candle["4. close"])

    # P1 is red, P2 is green
    if not (prev_close < prev_open and current_close > current_open):
        return False

    return current_open > prev_close and current_close < prev_open


def is_bearish_harami(prev_candle, current_candle):
    prev_open = float(prev_candle["1. open"])
    prev_close = float(prev_candle["4. close"])
    current_open = float(current_candle["1. open"])
    current_close = float(current_candle["4. close"])

    # P1 is green, P2 is red
    if not (prev_close > prev_open and current_close < current_open):
        return False

    return current_open < prev_close and current_close > prev_open


In [39]:
def detect_harami(data, trend_days=3):
    time_series = data.get("Time Series (Daily)", {})
    sorted_dates = sorted(time_series.keys())
    results = []

    for i in range(trend_days + 1, len(sorted_dates)):
        current_date = sorted_dates[i]
        previous_date = sorted_dates[i - 1]
        current_candle = time_series[current_date]
        previous_candle = time_series[previous_date]
        trend_dates = sorted_dates[i - trend_days - 1 : i - 1]
        trend_candles = [time_series[d] for d in trend_dates]

        pattern = None
        stoploss = None
        entry_price = float(current_candle["4. close"])

        if is_downtrend(trend_candles) and is_bullish_harami(previous_candle, current_candle):
            pattern = "bullish_harami"
            stoploss = min(float(previous_candle["3. low"]), float(current_candle["3. low"]))

        elif is_uptrend(trend_candles) and is_bearish_harami(previous_candle, current_candle):
            pattern = "bearish_harami"
            stoploss = max(float(previous_candle["2. high"]), float(current_candle["2. high"]))

        if pattern:
            results.append({
                "date": current_date,
                "pattern": pattern,
                "entry_price": entry_price,
                "stoploss": stoploss,
                "open": float(current_candle["1. open"]),
                "high": float(current_candle["2. high"]),
                "low": float(current_candle["3. low"]),
                "close": float(current_candle["4. close"]),
                "volume": int(current_candle["5. volume"])
            })

    return pd.DataFrame(results)


In [40]:
def calculate_profit_loss_with_target_and_window(data, harami_df, target_pct=2.0, hold_days=3):
    time_series = data.get("Time Series (Daily)", {})
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df = df[['1. open', '2. high', '3. low', '4. close', '5. volume']].astype(float)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    trades = []
    total_profit_loss = 0.0

    for _, row in harami_df.iterrows():
        entry_date = pd.to_datetime(row["date"])
        direction = row["pattern"]
        entry_price = row["entry_price"]

        if entry_date not in df.index:
            continue

        future_dates = df.index[df.index > entry_date]
        window_dates = future_dates[:hold_days]

        if len(window_dates) == 0:
            continue

        target_price = (
            entry_price * (1 + target_pct / 100)
            if direction == "bullish_harami"
            else entry_price * (1 - target_pct / 100)
        )

        exit_price = None
        exit_date = None
        reason = ""

        # Step 1: Check for target hit in window
        for day in window_dates:
            high = df.loc[day]["High"]
            low = df.loc[day]["Low"]
            open_price = df.loc[day]["Open"]

            if direction == "bullish_harami" and high >= target_price:
                exit_price = open_price
                exit_date = day
                reason = "target_hit"
                break
            elif direction == "bearish_harami" and low <= target_price:
                exit_price = open_price
                exit_date = day
                reason = "target_hit"
                break

        # Step 2: If target not hit, exit at max/min within window
        if exit_price is None:
            if direction == "bullish_harami":
                max_day = df.loc[window_dates]["High"].idxmax()
                exit_price = df.loc[max_day]["High"]
                exit_date = max_day
                reason = "max_window_exit"
            elif direction == "bearish_harami":
                min_day = df.loc[window_dates]["Low"].idxmin()
                exit_price = df.loc[min_day]["Low"]
                exit_date = min_day
                reason = "min_window_exit"

        if exit_price is None or exit_date is None:
            continue

        profit_loss = (
            exit_price - entry_price if direction == "bullish_harami"
            else entry_price - exit_price
        )
        total_profit_loss += profit_loss

        trades.append({
            'entry_date': entry_date,
            'entry_price': entry_price,
            'exit_date': exit_date,
            'exit_price': exit_price,
            'target_price': target_price,
            'direction': direction,
            'profit_loss': profit_loss,
            'exit_reason': reason,
            'stoploss': row.get("stoploss", None)
        })

    return total_profit_loss, trades


In [41]:
def plot_candlestick(data):
    # Extract the time series data and convert it into a DataFrame
    time_series = data.get("Time Series (Daily)", {})

    # Convert the time series into a DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')

    # Convert all values to float and rename columns for mplfinance compatibility
    df = df[['1. open', '2. high', '3. low', '4. close', '5. volume']].astype(float)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    # Convert the index to datetime
    df.index = pd.to_datetime(df.index)
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

    fig.show()


In [50]:
harami_df = detect_harami(data)
total_profit_loss, trades = calculate_profit_loss_with_target_and_window(
    data,
    harami_df,
    target_pct=2.0,
    hold_days=3
)

print(f"Total Profit/Loss: {total_profit_loss:.2f}")
for trade in trades:
    print(
        f"{trade['direction']} | Entry: {trade['entry_date'].date()} at {trade['entry_price']:.2f}, "
        f"Exit: {trade['exit_date'].date()} at {trade['exit_price']:.2f} "
        f"(Target: {trade['target_price']:.2f}, Reason: {trade['exit_reason']}), "
        f"P/L: {trade['profit_loss']:.2f}"
    )


Total Profit/Loss: 0.12
bearish_harami | Entry: 2025-01-03 at 45.78, Exit: 2025-01-06 at 45.66 (Target: 44.86, Reason: target_hit), P/L: 0.12


In [51]:
plot_candlestick(data)